Скачивание и парсинг данных о книгах и читателях с сайта www.livelib.ru

In [17]:
import numpy as np
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup
from html.parser import HTMLParser
import string
from selenium import webdriver
import html
from time import sleep
from selenium.common.exceptions import NoSuchElementException
from pandas import DataFrame

In [18]:
way_driver = "D:\Program\chromedriver.exe"
syte_url = "https://www.livelib.ru"
readers_info = []
books_info = []

In [7]:
readers = []
for i in range(1, 5):
    readers_url = syte_url+"/readers/rating/listview/wide/~"+str(i)
    readers_html = requests.get(readers_url).text
    soup = BeautifulSoup(readers_html, 'html.parser')
    readers += list(map(lambda l: l.text, soup.find_all('a', class_='urow-login')))

In [15]:
def get_reader_info(driver, reader_name, t_sleep=1):
    reader_url = syte_url+"/reader/"+reader_name
    reader_html = requests.get(reader_url).content
    soup = BeautifulSoup(reader_html, 'html.parser')
    driver.get(reader_url)
    sleep(t_sleep)
    result = []
    reader_info = {
        'login': '0',
        'gender': '-',
        'rating': '0',
        'marks_books': []
    }
    reader_info['login'] = reader_name
    items = driver.find_elements_by_class_name("group-row-title")
    for item in items:
        s = item.text
        if s.find('Пол:') == 0:
            reader_info['gender'] = s[5:len(s)]
        elif s.find('Рейтинг:') == 0:
            reader_info['rating'] = s[9:len(s)]
    sleep(t_sleep)
    #Прочитанные книги
    reader_url = syte_url+"/reader/"+reader_name+"/read"
    reader_html = requests.get(reader_url).content
    soup = BeautifulSoup(reader_html, 'html.parser')
    driver.get(reader_url)
    sleep(t_sleep)
    number_item = driver.find_element_by_xpath("//*[@class='column-670 subcontainer on-left']")
    s = number_item.text
    s_number = ''
    i = 0
    while (s[i] < '0') or (s[i] > '9'):
        i = i + 1
    while (s[i] >= '0') and (s[i] <= '9'):
        s_number = s_number + s[i]
        i = i + 1
    number = int(s_number)
    number_page = number // 20
    if number % 20:
        number_page = number_page + 1
    for i in range(1, number_page + 1):
        reader_url = syte_url+"/reader/"+reader_name+"/read/~"+str(i)
        reader_html = requests.get(reader_url).content
        soup = BeautifulSoup(reader_html, 'html.parser')
        driver.get(reader_url)
        sleep(t_sleep)
        items_books = driver.find_elements_by_xpath("//*[@class='brow-book-name with-cycle']")
        items_marks = driver.find_elements_by_class_name("brow-ratings")
        for item_book, item_mark in zip(items_books, items_marks):
            mark_book = {
                'href': '',
                'mark': '',
            }
            mark_book['href'] = item_book.get_attribute("href")
            s = item_mark.text
            s_mark = ''
            i = 0
            while (s[i] < '0') or (s[i] > '9'):
                i = i + 1
            while (s[i] >= '0') and (s[i] <= '9'):
                s_mark = s_mark + s[i]
                i = i + 1
            if s[i] == '.':
                s_mark = s_mark + s[i]
                i = i + 1
                while (s[i] >= '0') and (s[i] <= '9'):
                    s_mark = s_mark + s[i]
                    i = i + 1
            mark_book['mark'] = s_mark
            reader_info['marks_books'].append(mark_book)
    #Рецензии
    reader_url = syte_url+"/reader/"+reader_name+"/reviews"
    reader_html = requests.get(reader_url).content
    soup = BeautifulSoup(reader_html, 'html.parser')
    driver.get(reader_url)
    sleep(t_sleep)
    number_item = driver.find_element_by_xpath('//*[@style="margin-right:50px;"]')
    s = number_item.text
    s_number = ''
    i = 0
    while (s[i] < '0') or (s[i] > '9'):
        i = i + 1
    while (s[i] >= '0') and (s[i] <= '9'):
        s_number = s_number + s[i]
        i = i + 1
    number = int(s_number)
    number_page = number // 25
    if number % 25:
        number_page = number_page + 1
    for i in range(1, number_page + 1):
        reader_url = syte_url+"/reader/"+reader_name+"/reviews/~"+str(i)
        reader_html = requests.get(reader_url).content
        soup = BeautifulSoup(reader_html, 'html.parser')
        driver.get(reader_url)
        sleep(t_sleep)
        items_books = driver.find_elements_by_class_name("book_name")
        items_marks = driver.find_elements_by_class_name("rating-book")
        for item_book, item_mark in zip(items_books, items_marks):
            mark_book = {
                'href': '',
                'mark': '',
            }
            mark_book['href'] = item_book.get_attribute("href")
            flag = true
            mark_book['mark'] = item_mark.text
            for j in reader_info['marks_books']:
                if j['href'] == mark_book['href']:
                    flag = false
            if flag:
                reader_info['marks_books'].append(mark_book)
    result.append(reader_info)
    return result

In [16]:
with webdriver.Chrome(way_driver) as driver:
    print(get_reader_info(driver, 'janekukeeva', 2))

[{'login': 'janekukeeva', 'gender': 'женский', 'rating': '618 баллов', 'marks_books': [{'href': 'https://www.livelib.ru/book/1001297574-dzhejn-ejr-sharlotta-bronte', 'mark': '3'}, {'href': 'https://www.livelib.ru/book/1002783459-razrushennyj-dvorets-erin-uatt', 'mark': '3'}, {'href': 'https://www.livelib.ru/book/1002743465-slomlennyj-prints-erin-uatt', 'mark': '3'}, {'href': 'https://www.livelib.ru/book/1002469768-bumazhnaya-printsessa-erin-uatt', 'mark': '4'}, {'href': 'https://www.livelib.ru/book/1000775899-a-zori-zdes-tihie-boris-vasilev', 'mark': '5'}, {'href': 'https://www.livelib.ru/work/1000253728-legkoe-dyhanie-ivan-bunin', 'mark': '2'}, {'href': 'https://www.livelib.ru/book/1002275721-dom-duhov-izabel-alende', 'mark': '3'}, {'href': 'https://www.livelib.ru/book/1002749981-vtoraya-zhizn-uve-fredrik-bakman', 'mark': '4'}, {'href': 'https://www.livelib.ru/book/1000292042-beguschij-za-vetrom-haled-hossejni', 'mark': '4'}, {'href': 'https://www.livelib.ru/book/1000667439-oblasti-tm

In [ ]:
with webdriver.Chrome(way_driver) as driver:
    for reader in readers:
        result = get_reader_info(driver, reader, 2)